In [ ]:
import pandas as pd
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sys.path.append("../../")
from utils.data_processing import _drop_consecutive_nans, add_day_ahead_column
from utils.error_metrics import _calc_mae, _calc_mse, _calc_rmse, _calc_nrmse, _calc_mape, _calc_mase, _calc_msse, _seas_naive_fcst, _calc_metrics
from utils.clustering import mapping_tsfeatures, clustering, sum_until_threshold, mapping_energy_metrics

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# PLOTTING
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 13  # Font size
stanford_colors = ['#1f78b5', '#33a12c', '#e41a1c', '#ff7f00', '#6a3d9b', '#b25928', #dark
                   '#a7cfe4', '#b3e08b', '#fc9b9a', '#fec06f', '#cbb3d7', '#ffff9a'] #light
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=stanford_colors)

# ML AZURE
from azureml.core import Workspace, Dataset, Datastore
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mlflow
from config import subscription_id, resource_group, workspace_name
workspace = Workspace(subscription_id, resource_group, workspace_name)
datastore = Datastore.get(workspace, "workspaceblobstore")

# 1. Forecast results - clustered & aggregated bus

In [ ]:
### Download data

name_list = [f'0{j}_load_bus_after' for j in range(0,10)]
forecast_test_dict = {}
metrics_test_dict = {}

for i, name in enumerate(name_list):
    path = f'azureml/{name}/results/'
    sec_char = name[1]
    model_name = 'model ' + sec_char

    print(f"Starting {model_name}")

    file = 'forecasts_forecast_test.csv'
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, path + file))
    forecast_test = dataset.to_pandas_dataframe() 
    forecast_test['ds'] = pd.to_datetime(forecast_test['ds'])
    print('IDs:', forecast_test['ID'].nunique())
    forecast_test_dict[model_name] = forecast_test

    file = 'metrics_metrics_test.csv'
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, path + file))
    metrics_test = dataset.to_pandas_dataframe()
    metrics_test_dict[model_name] = metrics_test

    print(f"Finished {model_name}")

In [ ]:
### Add all clusters together

metrics_all = pd.DataFrame()
forecast_test_all = pd.DataFrame()

for model_name, metrics_test in metrics_test_dict.items():
    metrics_test['cluster'] = model_name
    metrics_test['cluster'] = metrics_test['cluster'].str.extract(r'model (\d+)').astype(int)

    metrics_all = pd.concat([metrics_all, metrics_test.reset_index(drop=True)], ignore_index=True)
    print(model_name)

print('Starting forecast_test_all')
for model_name, forecast_test in forecast_test_dict.items():
    forecast_test['cluster'] = model_name
    forecast_test['cluster'] = forecast_test['cluster'].str.extract(r'model (\d+)').astype(int)
    
    forecast_test_all = pd.concat([forecast_test_all, forecast_test.reset_index(drop=True)], ignore_index=True)
    print(model_name)

In [ ]:
### Add yhat column

forecast_test_all_da = pd.DataFrame()
forecast_test_all_da = add_day_ahead_column(forecast_test_all, 'yhat')
forecast_test_all_da = forecast_test_all_da.dropna(subset=['yhat_day_ahead']).reset_index(drop=True)
forecast_test_all_da['residuals'] = forecast_test_all_da['y'] - forecast_test_all_da['yhat_day_ahead']
forecast_test_all_da.to_csv('07_cluster_forecasts_da.csv', index=False)

# 2. Histogram - MASE of all nodes

In [ ]:
fig = px.histogram(metrics_all, x="MASE", nbins=500, marginal="box", title='Distribution of MASE')
fig.update_layout(xaxis_title='MASE', yaxis_title='Count')
fig.update_xaxes(range=[0, 1])
fig.show()

# 3. Error on country-level (aggregated for comparison with country-level forecasts)

In [ ]:
forecast_test_all_da = pd.read_csv('07_cluster_forecasts_da.csv')
forecast_test_all_da['ID'] = forecast_test_all_da['ID'].astype(str)

In [ ]:
### Add country column

forecast_test_all_da['country'] = forecast_test_all_da['ID'].apply(lambda x: x.split('_')[0] if '_' in x else None)

mapping = pd.read_csv('02_mapping.csv')
mapping['ID'] = mapping['ID'].astype(str)
mapping['ID'] = mapping['ID'].astype(str)
temp = mapping[['ID', 'country']]
merged_df = forecast_test_all_da.merge(temp, on='ID', how='left')
forecast_test_all_da['country'] = merged_df['country_y'].fillna(merged_df['country_x'])

In [ ]:
### Calculate metrics for each country

forecast_test_country_agg = pd.DataFrame()
metrics_country = pd.DataFrame()
temp = forecast_test_all_da[['ds', 'y', 'ID', 'yhat_day_ahead', 'country']]

list = temp['country'].unique()

for country in list:
    string = f'{country}'

    filtered_rows_country = temp[temp['country'] == string]

    filtered_rows_country = filtered_rows_country.groupby(['ds']).sum().reset_index()
    filtered_rows_country['ID'] = string

    mae = _calc_mae(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'])
    mse = _calc_mse(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'])
    rmse = _calc_rmse(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'])
    mape = _calc_mape(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'])

    filtered_rows_country['snaive'] = filtered_rows_country['y'].shift(48)

    mase = _calc_mase(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'], snaive_predictions=filtered_rows_country['snaive'])
    msse = _calc_msse(predictions=filtered_rows_country['yhat_day_ahead'], truth=filtered_rows_country['y'], snaive_predictions=filtered_rows_country['snaive'])

    new_row = {'ID':string, 'RMSE':rmse, 'MAE':mae, 'MAPE':mape, 'MASE':mase, 'MSSE':msse}
    metrics_country = pd.concat([metrics_country, pd.DataFrame([new_row])])
    
    forecast_test_country_agg = pd.concat([forecast_test_country_agg, filtered_rows_country], ignore_index=True)

## Average error metrics for all countries
metrics_country_avg = pd.DataFrame([{'RMSE': metrics_country['RMSE'].mean(), 'MAE': metrics_country['MAE'].mean(), 'MAPE': metrics_country['MAPE'].mean(), 'MASE': metrics_country['MASE'].mean(), 'MSSE': metrics_country['MSSE'].mean()}])
print('Average metrics for all Countries:')
metrics_country_avg

In [ ]:
metrics_knn = pd.read_csv('../01_Benchmarks/metrics_knn.csv')
metrics_arima = pd.read_csv('../01_Benchmarks/metrics_arima.csv')
metrics_xgboost = pd.read_csv('../01_Benchmarks/metrics_xgboost.csv')
metrics_snaive = pd.read_csv('../01_Benchmarks/metrics_snaive.csv')

In [ ]:
### Plot distribution of two error metrics for every model in two plots

error1 = 'MAE'
error2 = 'MASE'

fig, ax = plt.subplots(2, 1, figsize=(8, 6), layout="constrained")

colors = stanford_colors[0:4]

bp1 = ax[0].boxplot([metrics_country[error1], metrics_knn[error1], metrics_arima[error1], metrics_xgboost[error1]], vert=False, labels=['Proposed\nbus model', 'k-NN', 'ARIMA', 'XGBoost'], showfliers=False)
ax[0].set_xlabel(f'{error1} [MW]')

bp2 = ax[1].boxplot([metrics_country[error2], metrics_knn[error2], metrics_arima[error2], metrics_xgboost[error2]], vert=False, labels=['Proposed\nbus model', 'k-NN', 'ARIMA', 'XGBoost'], showfliers=False)
ax[1].set_xlabel(f'{error2} [%]')

plt.show()

In [ ]:
### Country plot - bus

import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world[world.continent == 'Europe']
europe.loc[europe['iso_a3'] == 'PRT', 'iso_a3'] = 'POR'
europe_merged = europe.merge(metrics_country, how='left', left_on='iso_a3', right_on='ID')

# Plot
color_country = 'lightgray'

fig, ax = plt.subplots(1, 2, figsize=(12, 8))

world.plot(ax=ax[0], color=color_country, edgecolor=color_country)
europe_merged_plot1 = europe_merged.plot(column='MASE', cmap='YlOrRd', linewidth=1.0, ax=ax[0], edgecolor=color_country, legend=False, vmin=0.2, vmax=1)

world.plot(ax=ax[1], color=color_country, edgecolor=color_country)
europe_merged_plot2 = europe_merged.plot(column='MAE', cmap='YlOrRd', linewidth=1.0, ax=ax[1], edgecolor=color_country, legend=False, vmin=0, vmax=3000)

ax[0].set_ylim(33, 60)
ax[0].set_xlim(-10, 30)
ax[0].set_xticks([])
ax[0].set_yticks([])

ax[1].set_ylim(33, 60)
ax[1].set_xlim(-10, 30)
ax[1].set_xticks([])
ax[1].set_yticks([])

cbar1 = fig.colorbar(europe_merged_plot1.get_children()[1], ax=ax[0], orientation='horizontal', pad=0.03)
cbar1.set_label('MASE [%]')

cbar2 = fig.colorbar(europe_merged_plot2.get_children()[1], ax=ax[1], orientation='horizontal', pad=0.03)
cbar2.set_label('MAE [MW]')

plt.show()

# 5. Cluster validation

In [ ]:
### Prepare mapping

mapping = pd.read_csv("02_mapping.csv", index_col=0)
mapping['new_ID'] = mapping['country'].astype(str) + '_cluster0' + mapping['cluster'].astype(str) + '_agg_no_' + mapping['aggregation_no'].astype(str)
mapping = mapping.rename(columns={'cluster': 'cluster_1'})

cluster_mapping = forecast_test_all_da.drop_duplicates(subset=['ID', 'cluster']).reset_index(drop=True)
cluster_mapping = cluster_mapping[['ID', 'cluster']]
temp = cluster_mapping.rename(columns={'ID': 'new_ID'})
mapping = mapping.merge(temp, on='new_ID', how='left')
mapping = mapping.rename(columns={'cluster': 'cluster_2'})

In [ ]:
### Calculate magnitude of each node based on train data - Output [ID, y_mean, relative_magnitude] - 1min

buss_train = pd.read_csv("01_load_bus.csv")
buss_train['ds'] = pd.to_datetime(buss_train['ds'])
buss_train['ID'] = buss_train['ID'].astype(str)
print("loaded train data")

buss_train = buss_train[buss_train['ds'] < '2014-01-01']
temp = buss_train[['ID', 'y']]
buss_train_mean = temp.groupby('ID').mean().reset_index()
buss_train_sum = buss_train_mean['y'].sum()
buss_train_mean['relative_magnitude'] = buss_train_mean['y'] / buss_train_sum
buss_train_mean = buss_train_mean.rename(columns={'y': 'y_mean'})
buss_train_mean['y_mean'] = buss_train_mean['y_mean'].abs()
buss_magnitude = buss_train_mean
buss_magnitude['ID'] = buss_magnitude['ID'].astype(str)

### Calculate the magnitude after aggregation - Output [new_ID, relative_magnitude_agg]
buss_magnitude_aggregated = pd.DataFrame()
helper = []

for index, row in mapping.iterrows():
    ids = row['ID_list'].split(';')
    new_id = row['new_ID']

    filtered_rows = buss_magnitude[buss_magnitude['ID'].isin(ids)]
    magnitude = filtered_rows['relative_magnitude'].sum()
    tempo = pd.DataFrame({'new_ID': [new_id], 'relative_magnitude_agg': [magnitude]})
    helper.append(tempo)

buss_magnitude_aggregated = pd.concat(helper).reset_index(drop=True)

temp = forecast_test_all_da[~forecast_test_all_da['ID'].isin(buss_magnitude_aggregated['new_ID'])]
temp = temp['ID'].unique()
temp = buss_magnitude[buss_magnitude['ID'].isin(temp)]
not_agg = temp[['ID', 'relative_magnitude']]
buss_magnitude_aggregated = pd.concat([buss_magnitude_aggregated, not_agg.rename(columns={'ID': 'new_ID', 'relative_magnitude': 'relative_magnitude_agg'})]).reset_index(drop=True)

### Calculate the magnitude for each cluster - Output [cluster, relative_magnitude_cluster]
buss_magnitude_cluster = buss_magnitude_aggregated.merge(cluster_mapping.rename(columns={'ID': 'new_ID'}), on='new_ID', how='left')
buss_magnitude_cluster = buss_magnitude_cluster.drop(columns=['new_ID'])
buss_magnitude_cluster = buss_magnitude_cluster.groupby('cluster').sum().reset_index().rename(columns={'relative_magnitude_agg': 'relative_magnitude_cluster'})
buss_magnitude_cluster = buss_magnitude_cluster.sort_values(by='relative_magnitude_cluster', ascending=False)

In [ ]:
### Calculate tsfeatures & energy metrics (em) - using tsfeatures package - might take a while

list = []
del list

temp = forecast_test_all_da[['ds', 'y', 'ID']]
temp['y'] = temp['y'].astype('float32')

## TSFeatures & absolute em
ID_emp_features_copy_ts = mapping_tsfeatures(df=temp, normalise=True, calulate_tsfeatures=True, calculate_em=True)
ID_emp_features_copy_ts = ID_emp_features_copy_ts.iloc[:, [0] + [1] + list(range(13, 27))]

## Relative em
ID_emp_features_copy_em = mapping_energy_metrics(df=temp, normalise=True)
ID_emp_features_copy_em = ID_emp_features_copy_em.iloc[:, [0] + list(range(14, 20))]

## Merge
ID_emp_features_copy = ID_emp_features_copy_ts.merge(ID_emp_features_copy_em, on='ID', how='left')

In [ ]:
### Further processing of features

ID_emp_features = ID_emp_features_copy.copy()

temp = forecast_test_all_da[['ID', 'cluster']]
temp = temp.drop_duplicates(subset=['ID', 'cluster']).reset_index(drop=True)

tempo = pd.DataFrame(columns=['cluster', 'size'])
tempo['size'] = temp.groupby('cluster').size().reset_index(drop=True)
tempo['cluster'] = tempo.index

magnitude = buss_magnitude_cluster
magnitude = magnitude.merge(tempo, on='cluster', how='left')
magnitude['Average Load Share'] = magnitude['relative_magnitude_cluster'] / magnitude['size']

ID_emp_features = cluster_mapping.merge(ID_emp_features, on='ID', how='left')
ID_emp_features = magnitude[['cluster', 'relative_magnitude_cluster', 'Average Load Share']].merge(ID_emp_features, on='cluster', how='left')
ID_emp_features = ID_emp_features[ ['ID'] + [ col for col in ID_emp_features.columns if col != 'ID' ] ]

## Scaling of Features - choose between Standardization, Min/Max Scaling, Percentile Rank
scale = 'percentile_rank'

if scale == 'standardize':
    scaler = StandardScaler()
    scaler.fit(ID_emp_features.iloc[:, 3:])
    ID_emp_features.iloc[:, 3:] = scaler.transform(ID_emp_features.iloc[:, 3:])

if scale == 'percentile_rank':
    ID_emp_features.iloc[:, 2:] = ID_emp_features.iloc[:, 2:].rank(pct=True, method='min')

if scale == 'min_max':
    scaler = MinMaxScaler()
    scaler.fit(ID_emp_features.iloc[:, 3:])
    ID_emp_features.iloc[:, 3:] = scaler.transform(ID_emp_features.iloc[:, 3:])

## Renaming
median_cluster_emp = ID_emp_features.drop(columns=['ID'])
median_cluster_emp = median_cluster_emp.groupby('cluster').median().reset_index()
median_cluster_emp = median_cluster_emp.drop(columns=['cluster'])

median_cluster_emp.index = median_cluster_emp.index.astype(str)
median_cluster_emp.index = ['Cluster ' + cluster.astype(str) + f'  #IDs: {len(temp[temp["cluster"] == cluster])}' for cluster in temp['cluster'].unique()]
median_cluster_emp = median_cluster_emp.drop(columns=['var', 'nperiods', 'seasonal_period'])

median_cluster_emp = median_cluster_emp.sort_values(by='relative_magnitude_cluster', ascending=False)
median_cluster_emp = median_cluster_emp.rename(columns={'relative_magnitude_cluster': 'Overall Load Share'})

median_cluster_emp = median_cluster_emp.rename(columns={
    'lumpiness': 'Lumpiness',
    'stability': 'Stability',
    'entropy': 'Entropy',
    'trend': 'Trend',
    'spike': 'Spike',
    'linearity': 'Linearity',
    'curvature': 'Curvature',
    'e_acf1': 'ACF1',
    'e_acf10': 'ACF10',
    'seasonal_strength': 'Seasonal Strength',
    'peak': 'Peak',
    'trough': 'Troughs',
    'rbase': '$r_{base}$',
    'rminmax': '$r_{minmax}$',
    'rm2w': '$r_{m2w}$',
    'rn2w': '$r_{n2w}$',
    're2w': '$r_{e2w}$',
    'rni2w': '$r_{ni2w}$',
    })

median_cluster_emp = median_cluster_emp.T

## Late ordering after Average Load Share - Comment put to order after Total Load Share
average_load_share_row = median_cluster_emp.loc['Average Load Share']
sorted_columns = average_load_share_row.sort_values(ascending=False).index
median_cluster_emp = median_cluster_emp[sorted_columns]

In [ ]:
### Prepare for subplot 0 - error of individual buses per cluster

metrics_bus_cluster = pd.DataFrame()
temp = forecast_test_all_da[['ds', 'y', 'ID', 'yhat_day_ahead', 'cluster']]

for cluster in temp['cluster'].unique():
    filtered_rows_cluster = temp[temp['cluster'] == cluster]

    for id in filtered_rows_cluster['ID'].unique():
        filtered_rows = filtered_rows_cluster[filtered_rows_cluster['ID'] == id]

        mae = _calc_mae(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'])
        mse = _calc_mse(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'])
        rmse = _calc_rmse(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'])
        mape = _calc_mape(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'])

        filtered_rows['snaive'] = filtered_rows['y'].shift(48)

        mase = _calc_mase(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'], snaive_predictions=filtered_rows['snaive'])
        msse = _calc_msse(predictions=filtered_rows['yhat_day_ahead'], truth=filtered_rows['y'], snaive_predictions=filtered_rows['snaive'])

        new_row = {'cluster': cluster, 'ID':id, 'RMSE':rmse, 'MAE':mae, 'MAPE':mape, 'MASE':mase, 'MSSE':msse}
        metrics_bus_cluster = pd.concat([metrics_bus_cluster, pd.DataFrame([new_row])])

metrics_bus_cluster = metrics_bus_cluster.merge(buss_magnitude_cluster, how='left', on='cluster')

temp = pd.DataFrame(columns=['cluster', 'size'])
temp['size'] = metrics_bus_cluster.groupby('cluster').size().reset_index(drop=True)
temp['cluster'] = temp.index
metrics_bus_cluster = metrics_bus_cluster.merge(temp, on='cluster', how='right')
metrics_bus_cluster['Average Load Share'] = metrics_bus_cluster['relative_magnitude_cluster'] / metrics_bus_cluster['size']
metrics_bus_cluster = metrics_bus_cluster.drop(columns=['size'])

grouped_data = metrics_bus_cluster.groupby('cluster')['MAE'].apply(list).reset_index(name='MAE')
grouped_data = grouped_data.merge(metrics_bus_cluster[['cluster', 'Average Load Share']], on='cluster', how='left')
grouped_data = grouped_data.sort_values(by='Average Load Share', ascending=False).reset_index(drop=True)
grouped_data = grouped_data.drop_duplicates(subset='cluster', kebuss='first').reset_index(drop=True)

mase_values = [values for values in grouped_data['MAE']]
cluster_labels = [cluster for cluster in grouped_data['cluster']]

In [ ]:
### Plot

import seaborn as sns

fig, axs = plt.subplots(4, 1, figsize=(10, 11), sharex=True, gridspec_kw={'height_ratios': [6, len(median_cluster_emp.iloc[:2].index), len(median_cluster_emp.iloc[2:14].index), len(median_cluster_emp.iloc[14:].index)]})

## Subplot 0 - Error Distribution
positions = range(len(cluster_labels))
positions = [pos + 0.5 for pos in positions]
axs[0].boxplot(mase_values, labels=cluster_labels, positions=positions, showfliers=False)
axs[0].set_title('Distribution of MAE Error Metric on Bus-Level by Cluster')
axs[0].set_ylabel('MAE [MW]', rotation=0, labelpad=45)

## Subplot 1 - Magnitude
heatmap1 = sns.heatmap(median_cluster_emp.iloc[:2], cmap='coolwarm', annot=False, fmt=".2f", linewidths=.5, annot_kws={"fontsize":8}, ax=axs[1], cbar=False)
axs[1].set_title('Load Share')
yticks_positions = [tick + 0.5 for tick in range(len(median_cluster_emp.iloc[:2].index))]  # Calculate middle of each row
heatmap1.set_yticks(yticks_positions)
heatmap1.set_yticklabels(median_cluster_emp.iloc[:2].index, rotation=0)

## Subplot 2 - TS
heatmap2 = sns.heatmap(median_cluster_emp.iloc[2:14], cmap='coolwarm', annot=False, fmt=".2f", linewidths=.5, annot_kws={"fontsize":8}, ax=axs[2], cbar=False)
axs[2].set_title('Time Series Characteristics')

## Subplot 3 - Energy
heatmap3 = sns.heatmap(median_cluster_emp.iloc[14:], cmap='coolwarm', annot=False, fmt=".2f", linewidths=.5, annot_kws={"fontsize":8}, ax=axs[3], cbar=False)
axs[3].set_title('Relative Energy Metrics')

## Add colorbar
cbar_ax = fig.add_axes([1.0, 0.45, 0.01, 0.3])   # [left, bottom, width, height]
cbar = fig.colorbar(heatmap2.collections[0], cax=cbar_ax)
cbar.set_label('Median Values of Percentile-Ranked Features')

plt.tight_layout()
plt.show()

# 6. Error plots

### 6.1 Residual bars

In [ ]:
### Preparing the inputs

## Top cluster to show - change for no. of cluster
no_cluster = 5

## Aggregation per Cluster - Whether to aggregate the data per cluster and timestamp or not
aggregation_per_cluster = False

## Tme stamp - maybe adjust
start_date = pd.to_datetime('2014-08-11')
end_date = start_date + pd.Timedelta(days=7)
forecast_test_all_da['ds'] = pd.to_datetime(forecast_test_all_da['ds'])
temp_df = forecast_test_all_da[(forecast_test_all_da['ds'] >= start_date) & (forecast_test_all_da['ds'] <= end_date)]
temp_df = temp_df[['ds', 'ID', 'y', 'yhat_day_ahead', 'residuals']]

temp_df['MAE'] = temp_df['residuals'].abs()
temp_df_agg = temp_df.copy()
temp_df_agg['residual_total'] = temp_df_agg['y'] - temp_df_agg['yhat_day_ahead']
temp_df_agg = temp_df_agg.drop(columns=['ID'])
temp_df_agg = temp_df_agg.groupby('ds').sum().reset_index()
temp_df = temp_df.merge(temp_df_agg[['ds', 'residual_total']], on='ds', how='left')
temp_df = temp_df.merge(cluster_mapping, on='ID', how='left')
temp_df = temp_df.merge(buss_magnitude_aggregated.rename(columns={'new_ID': 'ID'}), on='ID', how='left')
magnitude_sort = temp_df.groupby('cluster').agg({'relative_magnitude_agg': 'sum'}).sort_values(by='relative_magnitude_agg', ascending=False).reset_index()
top_cluster = magnitude_sort['cluster'].head(no_cluster).tolist()

## Aggregation per Cluster
if aggregation_per_cluster:
    temp_df = temp_df.drop(columns=['ID'])
    temp_df = temp_df.groupby(['ds', 'cluster']).sum().reset_index()
    temp_df = temp_df.sort_values(by='relative_magnitude_agg', ascending=False)

top_df = temp_df[temp_df['cluster'].isin(top_cluster)]
low_df = temp_df[~temp_df['cluster'].isin(top_cluster)]
low_df['cluster'] = np.where(low_df['residuals'] > 0, 'Other Positive', 'Other Negative')
temp_df = pd.concat([top_df, low_df], ignore_index=True)
grouped = temp_df.groupby(['ds', 'cluster']).residuals.sum().reset_index()
pivot_df = grouped.pivot(index='ds', columns='cluster', values='residuals').fillna(0)
data = pivot_df.values.T
data = data.astype(float)
data_shape = np.shape(data)

def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

In [ ]:
### Plot

plt.rcParams['font.size'] = 16

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

colors = stanford_colors[:no_cluster] + ['lightgray', 'lightgray']
pivot_df.columns = pivot_df.columns.astype(str)
pivot_df.columns = ['Cluster ' + col if col.isdigit() else col for col in pivot_df.columns]
component_labels = pivot_df.columns.tolist()
component_labels = ['Other' if x in ['Other Positive', 'Other Negative'] else x for x in component_labels]

## Bars
handles = []
labels = []
for i, label in enumerate(pivot_df.columns):
    bar = ax.bar(pivot_df.index, data[i], bottom=data_stack[i], width=0.03, label=component_labels[i], color=colors[i])
    handles.append(bar[0])
    labels.append(component_labels[i])

## Residual line
ax.plot(pivot_df.index, pivot_df.sum(axis=1), color='black', linewidth=2, label='Total')
handles.insert(0, ax.lines[0])
labels.insert(0, 'Total')

dates = pd.date_range(start_date, end_date, freq='1H')
custom_ticks = pd.date_range(start_date, end_date, freq='12H')
custom_labels = []

for tick in custom_ticks:
    if tick.hour == 0:
        custom_labels.append(tick.strftime('%a'))
    else:
        custom_labels.append(tick.strftime('%H:%M'))

ax.set_xticks(custom_ticks)
ax.set_xticklabels(custom_labels)
ax.legend(handles = handles[:-1], labels = labels[:-1], loc='upper left', bbox_to_anchor=(1, 1))
ax.set_ylabel('Residuals [MW]')
ax.set_xlabel('Date')
fig.show()

plt.rcParams['font.size'] = 13

In [ ]:
### Preparing Residuals

residuals = forecast_test_all_da[['ds', 'cluster', 'y', 'yhat_day_ahead', 'residuals']]
residuals = residuals.groupby(['ds', 'cluster']).sum().reset_index()
residuals = residuals[residuals['cluster'].isin(top_cluster)]
residuals['ds'] = pd.to_datetime(residuals['ds'])
residuals = residuals[(residuals['ds'] >= start_date) & (residuals['ds'] <= end_date)].reset_index(drop=True)

In [ ]:
### Plot

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

ax1 = axes[0]
colors = stanford_colors[:no_cluster] + ['lightgray', 'lightgray']
pivot_df.columns = pivot_df.columns.astype(str)
pivot_df.columns = ['Cluster ' + col if col.isdigit() else col for col in pivot_df.columns]
component_labels = pivot_df.columns.tolist()
component_labels = ['Other' if x in ['Other Positive', 'Other Negative'] else x for x in component_labels]

## Bars
handles = []
labels = []
for i, label in enumerate(pivot_df.columns):
    bar = ax1.bar(pivot_df.index, data[i], bottom=data_stack[i], width=0.03, label=component_labels[i], color=colors[i])
    handles.append(bar[0])
    labels.append(component_labels[i])

## Residual line
ax1.plot(pivot_df.index, pivot_df.sum(axis=1), color='black', linewidth=2, label='Total')
handles.insert(0, ax1.lines[0])
labels.insert(0, 'Total')

dates = pd.date_range(start_date, end_date, freq='1H')
custom_ticks = pd.date_range(start_date, end_date, freq='12H')
custom_labels = []

for tick in custom_ticks:
    if tick.hour == 0:
        custom_labels.append(tick.strftime('%a'))
    else:
        custom_labels.append(tick.strftime('%H:%M'))

ax1.set_xticks(custom_ticks)
ax1.set_xticklabels(custom_labels)
ax1.axhline(0, color='black', linestyle='--')
ax1.legend(handles = handles[:-1], labels = labels[:-1], loc='upper left', bbox_to_anchor=(1, 1))
ax1.set_ylabel('Residuals [MW]')

ax2 = axes[1]
ax2 = plt.gca()
temp = residuals['cluster'].unique()

for cluster in temp:
    data = residuals[residuals['cluster'] == cluster]
    ax2.plot(data['ds'], data['residuals'], linewidth=1, label=f'Cluster {cluster}')

ax2.set_xlabel('Date')
ax2.set_ylabel('Residuals [MW]')
ax2.axhline(0, color='black', linestyle='--')

fig.show()

### 6.2 Error share

In [ ]:
### Plotting preparation

temp_save = forecast_test_all_da[['ds', 'ID', 'y', 'yhat_day_ahead', 'residuals']]
temp_save = temp_save.merge(cluster_mapping, on='ID', how='left')
temp_save = temp_save.drop(columns=['ID'])
temp_save = temp_save.groupby(['ds', 'cluster']).sum().reset_index()

temp_save['residuals'] = temp_save['y'] - temp_save['yhat_day_ahead']
temp_save['MAE'] = temp_save['residuals'].abs()

## MAE Share - Subplot 1
total_error_mae = temp_save[['cluster', 'MAE']].groupby('cluster').mean().reset_index()
total_mae = total_error_mae['MAE'].sum()
total_error_mae['MAE_share'] = total_error_mae['MAE'] / total_mae

## Total Residual per Timestamp - Subplots 2 & 3
temp_df_agg = temp_save.copy()
temp_df_agg = temp_df_agg.groupby(['ds', 'cluster']).sum().reset_index()
temp_df_agg['residual_total'] = temp_df_agg['y'] - temp_df_agg['yhat_day_ahead']
temp_df_agg = temp_df_agg.groupby('ds').sum().reset_index()
temp_save = temp_save.merge(temp_df_agg[['ds', 'residual_total']], on='ds', how='left')

## Magnitude - Subplot 4
total_error_magnitude = temp_save[['cluster', 'y']].groupby('cluster').mean().reset_index()
total_magnitude = total_error_magnitude['y'].sum()
total_error_magnitude['magnitude_share'] = total_error_magnitude['y'] / total_magnitude

In [ ]:
### Preparation

quantile = 0.90

## Positive Share
result_df_pos = temp_save[temp_save['residual_total'] > 0]
threshold = result_df_pos['residual_total'].quantile(quantile)
result_df_pos = result_df_pos[result_df_pos['residual_total'] > threshold]
total_error_pos = result_df_pos.groupby('cluster').mean().reset_index()
total_error_pos['bias_pos'] = total_error_pos.apply(lambda row: row['residuals'] / row['residual_total'], axis=1)
total_error_pos['error_pos'] = total_error_pos.apply(lambda row: row['MAE'] / row['residual_total'], axis=1)

## Negative Share
temp_save['residual_total'] = temp_save['residual_total'] * -1
result_df_neg = temp_save[temp_save['residual_total'] > 0]
threshold = result_df_neg['residual_total'].quantile(quantile)
result_df_neg = result_df_neg[result_df_neg['residual_total'] > threshold]
temp_save['residual_total'] = temp_save['residual_total'] * -1
total_error_neg = result_df_neg.groupby('cluster').mean().reset_index()
total_error_neg['bias_neg'] = total_error_neg.apply(lambda row: row['residuals'] / row['residual_total'], axis=1)
total_error_neg['error_neg'] = total_error_neg.apply(lambda row: row['MAE'] / row['residual_total'], axis=1)

total_error = total_error_pos[['cluster', 'bias_pos', 'error_pos']].merge(total_error_neg[['cluster', 'bias_neg', 'error_neg']], on='cluster', how='left')
total_error = total_error.merge(total_error_mae[['cluster', 'MAE_share']], on='cluster', how='left')
total_error = total_error.merge(total_error_magnitude[['cluster', 'magnitude_share']], on='cluster', how='left')
total_error['cluster_names'] = total_error['cluster'].apply(lambda x: f'Cluster {x}')
total_error = total_error.sort_values(by='magnitude_share', ascending=False)

In [ ]:
### Plot

import matplotlib.gridspec as gridspec

bias = 'black'
error = 'lightgray'
points = 'gray'

fig = plt.figure(figsize=(16, 5))
gs = gridspec.GridSpec(1, 4, width_ratios=[0.5, 1, 1, 0.5], wspace=0.45)

## Subplot 1: horizontal bar plot with relative share of RES
ax1 = plt.subplot(gs[0])
ax1.barh(total_error['cluster_names'], total_error['MAE_share'], color='none', edgecolor=points, label='Relative Aggregated Residuals')
ax1.set_xlabel('Overall MAE Share')
ax1.set_yticks([])
ax1.tick_params(axis=u'both', which=u'both', length=0)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)

## Subplot 2: Vertical line at x=0 and negative bar plot
ax2 = plt.subplot(gs[1])
ax2.barh(total_error['cluster_names'], total_error['error_neg'], label='Mean Absolute Error', color=error)
ax2.barh(total_error['cluster_names'], abs(total_error['bias_neg']), label='Mean Error (Bias)', color=bias)
ax2.set_xlabel(f'Relative Contribution to {round(100-quantile*100)} Percent \n Highest Negative Residuals')
ax2.set_yticks([])
ax2.invert_xaxis()
ax2.grid(True, axis='x', color='lightgray')
ax2.tick_params(axis=u'both', which=u'both',length=0)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.legend(loc='upper left')

## Subplot 3: Horizontal bar plot with positive RES
ax3 = plt.subplot(gs[2])
ax3.barh(total_error['cluster_names'], total_error['error_pos'], label='Mean Absolute Error', color=error)
ax3.barh(total_error['cluster_names'], total_error['bias_pos'], label='Mean Error (Bias)', color=bias)
ax3.set_xlabel(f'Relative Contribution to {round(100-quantile*100)} Percent \n Highest Positive Residuals')
ax3.tick_params(axis=u'both', which=u'both',length=0)
ax3.grid(True, axis='x', color='lightgray')
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.legend(loc='upper right')

## Subplot 4: Horizontal bar plot with magnitude
ax4 = plt.subplot(gs[3])
ax4.barh(total_error['cluster_names'], total_error['magnitude_share'], color='none', edgecolor=points, label='Relative Magnitude')
ax4.set_xlabel('Overall Load Share')
ax4.set_yticks([])
ax4.tick_params(axis=u'both', which=u'both',length=0)
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.spines['bottom'].set_visible(False)

plt.tight_layout()
plt.show()

### 6.3 Residuals in one cluster

In [ ]:
## Choose how many clusters to show
cluster_to_show = 5

forecast_one_cluster = forecast_test_all_da[forecast_test_all_da['cluster'] == cluster_to_show]
print('Number of buses/nodes in cluster:', forecast_one_cluster['ID'].nunique())

In [ ]:
### Preparation

## Top buses to show
no_id = 10

## Aggregation per Cluster - Whether to aggregate the data per cluster and timestamp or not
aggregation_per_cluster = False

## Time Frame 
start_date = pd.to_datetime('2014-08-11')
end_date = start_date + pd.Timedelta(days=7)
forecast_one_cluster['ds'] = pd.to_datetime(forecast_one_cluster['ds'])
temp_df = forecast_one_cluster[(forecast_one_cluster['ds'] >= start_date) & (forecast_one_cluster['ds'] <= end_date)]
temp_df = temp_df[['ds', 'ID', 'y', 'yhat_day_ahead', 'residuals']]

temp_df['MAE'] = temp_df['residuals'].abs()
temp_df_agg = temp_df.copy()
temp_df_agg['residual_total'] = temp_df_agg['y'] - temp_df_agg['yhat_day_ahead']
temp_df_agg = temp_df_agg.groupby('ds').sum().reset_index()
temp_df = temp_df.merge(temp_df_agg[['ds', 'residual_total']], on='ds', how='left')
temp_df = temp_df.merge(cluster_mapping, on='ID', how='left')
temp_df = temp_df.merge(buss_magnitude_aggregated.rename(columns={'new_ID': 'ID'}), on='ID', how='left')
magnitude_sort = temp_df.groupby('ID').agg({'relative_magnitude_agg': 'sum'}).sort_values(by='relative_magnitude_agg', ascending=False).reset_index()
top_id = magnitude_sort['ID'].head(no_id).tolist()

if aggregation_per_cluster:
    temp_df = temp_df.drop(columns=['ID'])
    temp_df = temp_df.groupby(['ds', 'cluster']).sum().reset_index()
    temp_df = temp_df.sort_values(by='relative_magnitude_agg', ascending=False)

top_df = temp_df[temp_df['ID'].isin(top_id)]
low_df = temp_df[~temp_df['ID'].isin(top_id)]
low_df['ID'] = np.where(low_df['residuals'] > 0, 'Other Positive', 'Other Negative')
temp_df = pd.concat([top_df, low_df], ignore_index=True)

grouped = temp_df.groupby(['ds', 'ID']).residuals.sum().reset_index()
pivot_df = grouped.pivot(index='ds', columns='ID', values='residuals').fillna(0)
data = pivot_df.values.T
data = data.astype(float)
data_shape = np.shape(data)

def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

In [ ]:
### Plot

fig, ax = plt.subplots(1, 1, figsize=(16, 5))
colors = stanford_colors[:no_id] + ['lightgray', 'lightgray']

pivot_df.columns = pivot_df.columns.astype(str)
pivot_df.columns = [x if x in ['Other Positive', 'Other Negative'] else 'L-bus ' + chr(ord('A') + i) for i, x in enumerate(pivot_df.columns)]
component_labels = pivot_df.columns.tolist()
component_labels = ['Other' if x in ['Other Positive', 'Other Negative'] else x for x in component_labels]

## Plot bars
handles = []
labels = []
for i, label in enumerate(pivot_df.columns):
    bar = ax.bar(pivot_df.index, data[i], bottom=data_stack[i], width=0.03, label=component_labels[i], color=colors[i])
    handles.append(bar[0])
    labels.append(component_labels[i])

## Plot residual line
ax.plot(pivot_df.index, pivot_df.sum(axis=1), color='black', linewidth=2, label='Total')
handles.insert(0, ax.lines[0])
labels.insert(0, 'Total')

dates = pd.date_range(start_date, end_date, freq='1H')
custom_ticks = pd.date_range(start_date, end_date, freq='12H')
custom_labels = []

for tick in custom_ticks:
    if tick.hour == 0:
        custom_labels.append(tick.strftime('%a'))
    else:
        custom_labels.append(tick.strftime('%H:%M'))

ax.set_xticks(custom_ticks)
ax.set_xticklabels(custom_labels)
ax.legend(handles = handles[:-1], labels = labels[:-1], loc='upper left', bbox_to_anchor=(1, 1))
ax.set_ylabel('Residuals [MW]')
ax.set_xlabel('Date')

fig.show()

### 6.4 Bus share

In [ ]:
### Preparation

temp_save = forecast_test_all_da[['ds', 'ID', 'y', 'yhat_day_ahead']]
temp_save['ds'] = pd.to_datetime(temp_save['ds'])
temp_save['ID'] = temp_save['ID'].astype(str)
temp_save = temp_save.groupby(['ds', 'ID']).sum().reset_index()
temp_save = temp_save.merge(cluster_mapping, on='ID', how='left')
temp_save['residuals'] = temp_save['y'] - temp_save['yhat_day_ahead']

temp_save['MAE'] = temp_save['residuals'].abs()
total_error_mae = temp_save[['ID', 'MAE']].groupby('ID').mean().reset_index()
total_mae = total_error_mae['MAE'].sum()
total_error_mae['MAE_share'] = total_error_mae['MAE'] / total_mae
temp_df_agg = temp_save.copy()
temp_df_agg = temp_df_agg.groupby(['ds', 'ID']).sum().reset_index()
temp_df_agg['residual_total'] = temp_df_agg['y'] - temp_df_agg['yhat_day_ahead']

temp_df_agg = temp_df_agg.groupby('ds').sum().reset_index()
temp_save = temp_save.merge(temp_df_agg[['ds', 'residual_total']], on='ds', how='left')

In [ ]:
### Preparation

sortierung = 'mae_share'
quantile = 0.90

## Positive share
result_df_pos = temp_save[temp_save['residual_total'] > 0]
threshold = result_df_pos['residual_total'].quantile(quantile)
result_df_pos = result_df_pos[result_df_pos['residual_total'] > threshold]

total_error_pos = result_df_pos.groupby('ID').mean().reset_index()

total_error_pos['bias_pos'] = total_error_pos.apply(lambda row: row['residuals'] / row['residual_total'], axis=1)
total_error_pos['error_pos'] = total_error_pos.apply(lambda row: row['MAE'] / row['residual_total'], axis=1)

## Negative share
temp_save['residual_total'] = temp_save['residual_total'] * -1
result_df_neg = temp_save[temp_save['residual_total'] > 0]
threshold = result_df_neg['residual_total'].quantile(quantile)
result_df_neg = result_df_neg[result_df_neg['residual_total'] > threshold]
temp_save['residual_total'] = temp_save['residual_total'] * -1

total_error_neg = result_df_neg.groupby('ID').mean().reset_index()

total_error_neg['bias_neg'] = total_error_neg.apply(lambda row: row['residuals'] / row['residual_total'], axis=1)
total_error_neg['bias_neg'] = total_error_neg['bias_neg'] * (-1)
total_error_neg['error_neg'] = total_error_neg.apply(lambda row: row['MAE'] / row['residual_total'], axis=1)

## Merge all together
total_error = total_error_pos[['ID', 'bias_pos', 'error_pos']].merge(total_error_neg[['ID', 'bias_neg', 'error_neg']], on='ID', how='left')
total_error = total_error.merge(total_error_mae[['ID', 'MAE_share']], on='ID', how='left')

## Change bus names 
total_error = total_error.merge(cluster_mapping, on='ID', how='left')
total_error = total_error.merge(buss_magnitude_aggregated.rename(columns={'new_ID': 'ID'}), on='ID', how='left')
total_error = total_error.sort_values(by='relative_magnitude_agg', ascending=False).reset_index(drop=True)
total_error['rank'] = total_error.groupby('cluster')['relative_magnitude_agg'].rank(ascending=False, method='first').astype(int)
total_error['ID_names'] = 'L-bus ' + total_error['rank'].astype(str).apply(lambda x: chr(64 + int(x))) + ' - Cluster ' + total_error['cluster'].astype(str)
total_error_before_sorting = total_error.copy()

if sortierung == 'bias_pos':
    total_error = total_error.sort_values(by='bias_pos', ascending=False)
if sortierung == 'bias_neg':
    total_error = total_error.sort_values(by='bias_neg', ascending=False)
if sortierung == 'mae_share':
    total_error = total_error.sort_values(by='MAE_share', ascending=False)

## Extract the top rows
total_error = total_error.head(10).reset_index(drop=True)

In [ ]:
### Plot

import matplotlib.gridspec as gridspec

bias = 'black'
error = 'lightgray'
points = 'gray'

fig = plt.figure(figsize=(17, 4))
gs = gridspec.GridSpec(1, 4, width_ratios=[0.9, 1.2, 1.2, 0.9], wspace=0.95)

## Subplot 1: horizontal bar plot with relative share of RES
ax1 = plt.subplot(gs[0])
ax1.barh(total_error['ID_names'], total_error['MAE_share'], color='none', edgecolor=points, label='Relative Aggregated Residuals')
ax1.set_xlabel('Overall MAE Share')
ax1.set_yticks([])
ax1.tick_params(axis=u'both', which=u'both', length=0)
ax1.set_xlim(0, 0.025)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)

## Subplot 2: Vertical line at x=0 and negative bar plot
ax2 = plt.subplot(gs[1])
ax2.barh(total_error['ID_names'], total_error['error_neg'], label='Mean Absolute Error', color=error)
ax2.barh(total_error['ID_names'], total_error['bias_neg'], label='Mean Error (Bias)', color=bias)
ax2.set_xlabel(f'Relative Contribution to {round(100-quantile*100)} Percent \n Highest Negative Residuals')
ax2.set_yticks([])
ax2.invert_xaxis()
ax2.grid(True, axis='x', color='lightgray')
ax2.tick_params(axis=u'both', which=u'both',length=0)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.legend(loc='upper left', bbox_to_anchor=(-0.75, 1.0))

## Subplot 3: Horizontal bar plot with positive RES
ax3 = plt.subplot(gs[2])
ax3.barh(total_error['ID_names'], total_error['error_pos'], label='Mean Absolute Error', color=error)
ax3.barh(total_error['ID_names'], total_error['bias_pos'], label='Mean Error (Bias)', color=bias)
ax3.set_xlabel(f'Relative Contribution to {round(100-quantile*100)} Percent \n Highest Positive Residuals')
ax3.tick_params(axis=u'both', which=u'both',length=0)
ax3.grid(True, axis='x', color='lightgray')
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.legend(loc='upper right', bbox_to_anchor=(1.75, 1.0))

## Subplot 4: Horizontal bar plot with magnitude
ax4 = plt.subplot(gs[3])
ax4.barh(total_error['ID_names'], total_error['relative_magnitude_agg'], color='none', edgecolor=points, label='Relative Magnitude')
ax4.set_xlabel('Overall Load Share')
ax4.set_yticks([])
ax4.tick_params(axis=u'both', which=u'both',length=0)
ax4.set_xlim(0, 0.02)
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.spines['bottom'].set_visible(False)

plt.tight_layout()
plt.show()

# 7. Aggregate Magnitude

In [ ]:
### Calculate the absolute magnitude for pre- & post aggregation

buss_magnitude_abs = pd.DataFrame()
buss_magnitude_aggregated_abs = pd.DataFrame()
buss_train_mean = buss_train[['ID', 'y']].groupby('ID').mean().reset_index()
buss_train_mean['y'] = buss_train_mean['y'].abs()
buss_train_mean = buss_train_mean.rename(columns={'y': 'y_mean'})
buss_magnitude_abs = buss_train_mean 
buss_magnitude_aggregated_abs = pd.DataFrame()
helper = []

for index, row in mapping.iterrows():
    ids = row['ID_list'].split(';')
    new_id = row['new_ID']

    filtered_rows = buss_train_mean[buss_train_mean['ID'].isin(ids)]
    magnitude = filtered_rows['y_mean'].sum()
    tempo = pd.DataFrame({'new_ID': [new_id], 'magnitude_agg': [magnitude]})
    helper.append(tempo)

buss_magnitude_aggregated_abs = pd.concat(helper).reset_index(drop=True)
temp = forecast_test_all_da[~forecast_test_all_da['ID'].isin(buss_magnitude_aggregated_abs['new_ID'])]
temp = temp['ID'].unique()
temp = buss_magnitude_abs[buss_magnitude_abs['ID'].isin(temp)]
not_agg = temp[['ID', 'y_mean']]
buss_magnitude_aggregated_abs = pd.concat([buss_magnitude_aggregated_abs, not_agg.rename(columns={'ID': 'new_ID', 'y_mean': 'magnitude_agg'})]).reset_index(drop=True)

In [ ]:
### Plot

fig, axs = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

axs[0].hist(buss_magnitude_abs['y_mean'], bins=200, color='grey')
axs[0].set_ylabel('Number of Buses')

axs[1].hist(buss_magnitude_aggregated_abs['magnitude_agg'], bins=100, color='grey')
axs[1].set_ylabel('Number of L-Buses')
axs[1].set_xlabel('Average Load [MW]')
axs[1].set_xscale('linear') 
axs[0].set_xlim(0, 1000)

plt.tight_layout()
plt.show()